In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import keras
import tensorflow as tf
from os import makedirs
from os.path import join, exists, expanduser
from keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.applications.imagenet_utils import decode_predictions

In [2]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [3]:
from tensorflow.python.client import device_lib

device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 6715403877394422672
 xla_global_id: -1,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 7806648320
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 6269991857280092926
 physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 3080, pci bus id: 0000:0a:00.0, compute capability: 8.6"
 xla_global_id: 416903419]

In [4]:
img_des = pd.read_csv('D:\RMBI_MMKG_data\img_des.csv')
img_des

,LabelName,name,description
0,/m/027rn,Dominican Republic,The Dominican Republic is a country located on...
1,/m/017dcd,Mighty Morphin Power Rangers,Mighty Morphin Power Rangers is an American su...
2,/m/01sl1q,Michelle Rodriguez,Mayte Michelle Rodriguez is an American actres...
3,/m/0cnk2q,Australia national football team,The Australia men's national soccer team repre...
4,/m/02_j1w,Defender,"In the sport of association football, a defend..."
...,...,...,...
14179,/m/0dlj8q2,Le Monde's 100 Books of the Century,The 100 Books of the Century is a list of the ...
14180,/m/05zp8,Palace,"A palace is a grand residence, especially a ro..."
14181,/m/024030,Padma Vibhushan,The Padma Vibhushan is the second-highest civi...
14182,/m/02y_9cf,Ethiopian Empire,"The Ethiopian Empire, also formerly known by t..."


In [5]:
SIZE = 1000

In [6]:
img_des_trunc = img_des.head(SIZE)
img_des_trunc

,LabelName,name,description
0,/m/027rn,Dominican Republic,The Dominican Republic is a country located on...
1,/m/017dcd,Mighty Morphin Power Rangers,Mighty Morphin Power Rangers is an American su...
2,/m/01sl1q,Michelle Rodriguez,Mayte Michelle Rodriguez is an American actres...
3,/m/0cnk2q,Australia national football team,The Australia men's national soccer team repre...
4,/m/02_j1w,Defender,"In the sport of association football, a defend..."
...,...,...,...
995,/m/01b9ck,Darryl F. Zanuck,Darryl Francis Zanuck was an American film pro...
996,/m/011ywj,Gosford Park,Gosford Park is a 2001 satirical black comedy ...
997,/m/0325pb,Alpha Delta Pi,"Alpha Delta Pi, commonly known as ADPi, is an ..."
998,/m/0mnrb,Henrico County,"Henrico County, officially the County of Henri..."


In [7]:
y = []
x_img = []
import os

for i in range(SIZE):
    path_of_the_directory = os.path.join('D:\RMBI_MMKG_data\Bulk-Bing-Image-downloader', img_des['name'][i])
    ext = ('.jpg')
    for path, dirc, files in os.walk(path_of_the_directory):
        for name in files:
            if name.endswith(ext):
                
                img_path = os.path.join(path, name)
                try:                
                    img = image.load_img(img_path, target_size=(224, 224))
                    img = image.img_to_array(img)
                
                    x_img.append(img)
                    y.append(img_des['name'][i])
                except:
                    print(img_path)

C:\Users\Alex\anaconda3\envs\gpu\lib\site-packages\PIL\TiffImagePlugin.py:845: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 2. 
  warnings.warn(str(msg))


D:\RMBI_MMKG_data\Bulk-Bing-Image-downloader\World War II\img_1460.jpg
D:\RMBI_MMKG_data\Bulk-Bing-Image-downloader\Crystal Palace F.C.\img_2403.jpg
D:\RMBI_MMKG_data\Bulk-Bing-Image-downloader\Crystal Palace F.C.\img_2404.jpg
D:\RMBI_MMKG_data\Bulk-Bing-Image-downloader\United Nations\img_3556.jpg
D:\RMBI_MMKG_data\Bulk-Bing-Image-downloader\Marvel Entertainment\img_4278.jpg
D:\RMBI_MMKG_data\Bulk-Bing-Image-downloader\Billy Connolly\img_5103.jpg
D:\RMBI_MMKG_data\Bulk-Bing-Image-downloader\APOEL FC\img_5873.jpg
D:\RMBI_MMKG_data\Bulk-Bing-Image-downloader\Fayetteville\img_104019.jpg
D:\RMBI_MMKG_data\Bulk-Bing-Image-downloader\SG Dynamo Dresden\img_6375.jpg
D:\RMBI_MMKG_data\Bulk-Bing-Image-downloader\Shrek\img_6777.jpg
D:\RMBI_MMKG_data\Bulk-Bing-Image-downloader\Dan Povenmire\img_7498.jpg
D:\RMBI_MMKG_data\Bulk-Bing-Image-downloader\Election\img_57947.jpg


In [8]:
y = np.array(y)
x_img = np.array(x_img)

In [9]:
print(y.shape)
print(x_img.shape)

(8491,)
(8491, 224, 224, 3)


In [10]:
y

array(['Dominican Republic', 'Dominican Republic', 'Dominican Republic',
       ..., 'Theatrical producer', 'Theatrical producer',
       'Theatrical producer'], dtype='<U97')

In [11]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(y)
y_encoded = le.transform(y)

In [12]:
y_encoded

array([265, 265, 265, ..., 890, 890, 890])

In [13]:
from tensorflow.keras.utils import to_categorical
y_one_hot = to_categorical(y_encoded)
y_one_hot

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [14]:
y_one_hot.shape

(8491, 983)

In [15]:
from sklearn.model_selection import train_test_split
x_img_train, x_img_test, y_one_hot_train, y_one_hot_test, y_encoded_train, y_encoded_test = train_test_split(x_img, y_one_hot, y_encoded, test_size=0.2)

In [16]:
x_img_train

array([[[[255., 255., 255.],
         [255., 255., 255.],
         [255., 255., 255.],
         ...,
         [255., 255., 255.],
         [255., 255., 255.],
         [255., 255., 255.]],

        [[255., 255., 255.],
         [255., 255., 255.],
         [255., 255., 255.],
         ...,
         [255., 255., 255.],
         [255., 255., 255.],
         [255., 255., 255.]],

        [[255., 255., 255.],
         [255., 255., 255.],
         [255., 255., 255.],
         ...,
         [255., 255., 255.],
         [255., 255., 255.],
         [255., 255., 255.]],

        ...,

        [[255., 255., 255.],
         [255., 255., 255.],
         [255., 255., 255.],
         ...,
         [255., 255., 255.],
         [255., 255., 255.],
         [255., 255., 255.]],

        [[255., 255., 255.],
         [255., 255., 255.],
         [255., 255., 255.],
         ...,
         [255., 255., 255.],
         [255., 255., 255.],
         [255., 255., 255.]],

        [[255., 255., 255.],
       

In [17]:
x_img_train, x_img_test = x_img_train / 255.0, x_img_test / 255.0

In [18]:
x_img_train

array([[[[1.        , 1.        , 1.        ],
         [1.        , 1.        , 1.        ],
         [1.        , 1.        , 1.        ],
         ...,
         [1.        , 1.        , 1.        ],
         [1.        , 1.        , 1.        ],
         [1.        , 1.        , 1.        ]],

        [[1.        , 1.        , 1.        ],
         [1.        , 1.        , 1.        ],
         [1.        , 1.        , 1.        ],
         ...,
         [1.        , 1.        , 1.        ],
         [1.        , 1.        , 1.        ],
         [1.        , 1.        , 1.        ]],

        [[1.        , 1.        , 1.        ],
         [1.        , 1.        , 1.        ],
         [1.        , 1.        , 1.        ],
         ...,
         [1.        , 1.        , 1.        ],
         [1.        , 1.        , 1.        ],
         [1.        , 1.        , 1.        ]],

        ...,

        [[1.        , 1.        , 1.        ],
         [1.        , 1.        , 1.        ]

In [19]:
ResNet = ResNet50(weights="imagenet", include_top=True, input_shape=(224, 224, 3))

In [20]:
model = tf.keras.models.Sequential()
model.add(ResNet) 
model.add(tf.keras.layers.Dense(y_one_hot.shape[1], activation='softmax'))

In [21]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 1000)              25636712  
                                                                 
 dense (Dense)               (None, 983)               983983    
                                                                 
Total params: 26,620,695
Trainable params: 26,567,575
Non-trainable params: 53,120
_________________________________________________________________


In [22]:
model.compile(
    optimizer=tf.keras.optimizers.RMSprop(1e-3),
    loss=[
        tf.keras.losses.CategoricalCrossentropy(from_logits=True),
    ],
    metrics=['accuracy']
)

In [23]:
import time
start = time.time()
print("Fit model on training data")
history = model.fit(
    x_img_train,
    y_one_hot_train,
    batch_size=20,
    epochs=10,
)
end = time.time()
print("Required training time: %f s" % (end - start))

Fit model on training data
Epoch 1/10


C:\Users\Alex\anaconda3\envs\gpu\lib\site-packages\tensorflow\python\util\dispatch.py:1082: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


340/340 [==============================] - 34s 80ms/step - loss: 6.8957 - accuracy: 0.0021
Epoch 2/10
340/340 [==============================] - 26s 78ms/step - loss: 6.8767 - accuracy: 0.0044
Epoch 3/10
340/340 [==============================] - 27s 78ms/step - loss: 6.8617 - accuracy: 0.0044
Epoch 4/10
340/340 [==============================] - 27s 78ms/step - loss: 6.8361 - accuracy: 0.0044
Epoch 5/10
340/340 [==============================] - 27s 78ms/step - loss: 6.8057 - accuracy: 0.0046
Epoch 6/10
340/340 [==============================] - 27s 78ms/step - loss: 6.7720 - accuracy: 0.0050
Epoch 7/10
340/340 [==============================] - 27s 78ms/step - loss: 6.7326 - accuracy: 0.0050
Epoch 8/10
340/340 [==============================] - 26s 78ms/step - loss: 6.6921 - accuracy: 0.0047
Epoch 9/10
340/340 [==============================] - 27s 78ms/step - loss: 6.6542 - accuracy: 0.0057
Epoch 10/10
340/340 [==============================] - 27s 78ms/step - loss: 6.6165 - accurac

In [24]:
start = time.time()
results = model.evaluate(
    x_img_test, 
    y_one_hot_test, 
    batch_size=10)
end = time.time()
print("Required duration: %f s" % (end - start))

170/170 [==============================] - 3s 15ms/step - loss: 6.8612 - accuracy: 0.0071
Required duration: 3.403553 s


In [27]:
from sklearn.metrics import classification_report

y_pred = model.predict(x_img_test, batch_size=1, verbose=1)
y_pred_bool = np.argmax(y_pred, axis=1)

print(classification_report(y_encoded_test, y_pred_bool, digits = 6))
print("Required duration: %f s" % (end - start))

1699/1699 [==============================] - 11s 6ms/step
              precision    recall  f1-score   support

           0   0.000000  0.000000  0.000000         1
           1   0.000000  0.000000  0.000000         1
           3   0.000000  0.000000  0.000000         3
           4   0.000000  0.000000  0.000000         1
           5   0.000000  0.000000  0.000000         4
           6   0.000000  0.000000  0.000000         1
           7   0.000000  0.000000  0.000000         4
           8   0.000000  0.000000  0.000000         4
           9   0.000000  0.000000  0.000000         1
          10   0.000000  0.000000  0.000000         2
          11   0.000000  0.000000  0.000000         2
          12   0.000000  0.000000  0.000000         1
          13   0.000000  0.000000  0.000000         1
          14   0.000000  0.000000  0.000000         1
          15   0.000000  0.000000  0.000000         3
          16   0.000000  0.000000  0.000000         2
          17   0.000000

C:\Users\Alex\anaconda3\envs\gpu\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Alex\anaconda3\envs\gpu\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Alex\anaconda3\envs\gpu\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
model.save('Resnet_only_CNN.h5')